In [1]:
from src.flow_factory.hparams.args import Arguments

In [2]:
arg = Arguments.load_from_yaml("config/full/flux.yaml")

[2025-12-25 11:01:57,665] [INFO] [src.flow_factory.hparams.training_args]: World Size:1


In [3]:
arg

launcher: accelerate
config_file: config/accelerate_configs/deepspeed_zero2.yaml
num_processes: 8
main_process_port: 29500
mixed_precision: bf16
run_name: flux1_full_20251225_110157
project: Flow-Factory
logging_backend: wandb
data:
  dataset_dir: dataset/t2is
  preprocessing_batch_size: 8
  dataloader_num_workers: 16
  enable_preprocess: true
  force_reprocess: true
  max_dataset_size: 1000
model:
  model_name_or_path: black-forest-labs/FLUX.1-dev
  finetune_type: full
  master_weight_dtype: float32
  target_modules: default
  resume_path: null
  model_type: flux1
  lora_rank: 8
  lora_alpha: 16
training:
  resolution: !!python/tuple
  - 512
  - 512
  height: 512
  width: 512
  per_device_batch_size: 1
  gradient_step_per_epoch: 2
  max_grad_norm: 1.0
  num_batches_per_epoch: 768
  gradient_accumulation_steps: 384
  trainer_type: grpo
  clip_range: !!python/tuple
  - -0.0001
  - 0.0001
  adv_clip_range: !!python/tuple
  - -5.0
  - 5.0
  group_size: 16
  unique_sample_num_per_epoch: 48

In [6]:
import torch
from datasets import load_dataset
import numpy as np

In [5]:
any(i==2 for i in []), all(i is not None for i in [])

(False, True)

In [6]:
t = torch.randn(3, 4, 5)
t.unsqueeze(2).shape

torch.Size([3, 4, 1, 5])

In [17]:
rewards = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0]).numpy()
prompt_ids = torch.tensor([0, 2, 1, 1, 0, 2]).numpy()
unique_prompt_ids, group_indices = np.unique(prompt_ids, return_inverse=True, axis=0)
std = max(np.std(rewards, axis=0, keepdims=True), 1e-6)
advantages = np.zeros_like(rewards, dtype=np.float64)

In [20]:
for group_id in unique_prompt_ids:
    mask = (group_indices == group_id)
    group_rewards = rewards[mask]
    print(group_rewards)
    mean = np.mean(group_rewards, keepdims=True)
    advantages[mask] = (group_rewards - mean) / std
    print(advantages[mask])

[1. 5.]
[-1.17107999  1.17107999]
[3. 4.]
[-0.29277  0.29277]
[2. 6.]
[-1.17107999  1.17107999]


In [19]:
advantages

array([-1.17107999, -1.17107999, -0.29277   ,  0.29277   ,  1.17107999,
        1.17107999])